In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import re

In [2]:
model_dict = {"LoraInterp": "lora",
              "LoraCNInterp": "cn",
              "AnimeInterp": "aninterp"}

def get_model_abr(model):
    try:
        return model_dict[model]
    except:
        return None

In [3]:
root_path = "/cs/labs/danix/nadav_al/AnimeInterp/experiments/outputs"

In [4]:
ckpts = {"1200": 0, "2300": 1}

In [5]:
def extract_rank(string):
    pattern = r'rk(\d+)'
    match = re.search(pattern, string)
    if match:
        return "rk", match.group(1)
    else:
        return None, None

In [36]:
def read_dataframe(path):
    df = pd.read_csv(path)
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
    return df

In [37]:
def init_dataframes():
    dictionary = { 
        "aiw": {"lora": {"both": [None, None], "latents": [None, None], "result": [None, None]},
               "cn": {"both": [None, None], "latents": [None, None]},
               "aninterp": [None],
               },
        "aot": {"lora": {"both": [None, None], "latents": [None, None], "result": [None, None]},
               "cn": {"both": [None, None], "latents": [None, None]},
               "aninterp": [None],
               },
        "lk": {"lora": {"both": [None, None], "latents": [None, None], "result": [None, None]},
              "cn": {"both": [None, None], "latents": [None, None]},
              "aninterp": [None],
              },
        "sat": {"lora": {"both": [None, None], "latents": [None, None], "result": [None, None]},
               "cn": {"both": [None, None], "latents": [None, None]},
               "aninterp": [None],
               },
        "up": {"lora": {"both": [None, None], "latents": [None, None], "result": [None, None]},
              "cn": {"both": [None, None], "latents": [None, None]},
              "aninterp": [None],
              },
    }
    for movie in os.listdir(root_path):
        if not movie.lower() in dictionary.keys():
            continue
        print(movie)
        movie_path = os.path.join(root_path, movie)
        for model in os.listdir(movie_path):
            model_abr = get_model_abr(model)
            if not model_abr: continue
            elif model_abr == "lora" or model_abr == "cn":
                model_path = os.path.join(movie_path, model)
                for type in os.listdir(model_path):
                    type_path = os.path.join(model_path, type)
                    for ckpt in os.listdir(type_path):
                        ckpt_path = os.path.join(type_path, ckpt)
                        for test in os.listdir(ckpt_path):
                            _, rank_number = extract_rank(test)
                            psnr_path = os.path.join(ckpt_path, test, "psnr.csv")
                            if os.path.exists(psnr_path):
                                df = read_dataframe(psnr_path)
                                print(movie.lower(), model_abr, type)
                                if dictionary[movie.lower()][model_abr][type][ckpts[ckpt]] is None:
                                    dictionary[movie.lower()][model_abr][type][ckpts[ckpt]] = {}
                                dictionary[movie.lower()][model_abr][type][ckpts[ckpt]][rank_number] = df
            else:
                model_path = os.path.join(movie_path, model)
                psnr_path = os.path.join(model_path, "psnr.csv")
                if os.path.exists(psnr_path):
                    df = pd.read_csv(psnr_path)
                    if dictionary[movie.lower()][model] is None:
                        dictionary[movie.lower()][model] = []
                    dictionary[movie.lower()][model].append(df)
    return dictionary

In [38]:
dic = init_dataframes()

AIW
aiw lora latents
aiw lora latents
aiw lora latents
aiw lora latents
aiw lora both
aiw lora both
aiw lora both
aiw lora both
aiw lora result
aiw lora result
aiw lora result
aiw lora result
aiw cn latents
aiw cn latents
aiw cn latents
aiw cn latents
aiw cn both
aiw cn both
aiw cn both
aiw cn both
SAT
LK
lk lora latents
lk lora latents
lk lora latents
lk lora latents
lk lora both
lk lora both
lk lora both
lk lora both
lk lora both
lk lora result
lk lora result
lk lora result
lk lora result
lk cn latents
lk cn latents
lk cn latents
lk cn both
lk cn both
lk cn both
AoT
aot lora latents
aot lora latents
aot lora latents
aot lora latents
aot lora latents
aot lora latents
aot lora both
aot lora both
aot lora both
aot lora both
aot lora both
aot lora both
aot lora result
aot lora result
aot lora result
aot lora result
aot lora result
aot lora result
aot cn latents
aot cn latents
aot cn latents
aot cn latents
aot cn latents
aot cn latents
aot cn latents
aot cn both
aot cn both
aot cn both
ao

In [39]:
dic

{'aiw': {'lora': {'both': [None,
    {'10':      ValidIndex    Frame_2
     0             0  23.896101
     1             1  22.849476
     2             2  24.692820
     3             3  27.059944
     4             4  23.434177
     ..          ...        ...
     313         313  24.161921
     314         314  24.814209
     315         315  24.005455
     316         316  22.238606
     317         317  26.015236
     
     [318 rows x 2 columns],
     '4':      ValidIndex    Frame_2
     0             0  23.947071
     1             1  22.725559
     2             2  24.739944
     3             3  26.802245
     4             4  23.683396
     ..          ...        ...
     313         313  24.302889
     314         314  24.819784
     315         315  23.859154
     316         316  22.267739
     317         317  26.319332
     
     [318 rows x 2 columns],
     '6':      ValidIndex    Frame_2
     0             0  24.059000
     1             1  22.829005
     2           

In [29]:
ckpt_idx = ckpts[2300]

def plot_loss_models(dic):
    models_df_dict = {}
    for movie_title in dic:
        movie = dic[movie_title]
        for model_name in movie:
            dfs = []
            if model_name in [ 'lora', 'cn' ]:
                for objective in movie[model_name]:
                    model = movie[model_name][objective]
                    for _, df in model.items():
                        dfs.append(df)
            
            model_df = pd.concat(dfs)
                        
            
        
        

In [30]:
dict_to_dataframe(dic)

ValueError: 2